# Title: Representation 

## 1:   Imports

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import anndata as ad
# import scanpy as sc
from collections import Counter
os.environ['SCIPY_ARRAY_API'] = '1'
# from imblearn import over_sampling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from flaml import AutoML
import h5py
# import 

## 2: Load  metadata and set data dir

In [2]:
embed_dir = '/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/'
data = pd.read_csv('../data/processed/dataset_w_cdr_annotation.csv', index_col=0)


In [ ]:
# Extrac embeding

embed_to_load = ["8M","35M","150M"]
chain_type = ["together","seperate"]
linker_type = ["_<cls>","_G","_GGGGS"]
for embed in embed_to_load:
    for chains in chain_type:
        for linker in linker_type:
            if chains == "seperate":
                linker=""
            model_name = os.path.join(embed_dir, f'embed_EMS_{embed}_{chains}{linker}')
            compress_name = model_name+".h5"
            
            if os.path.exists(compress_name):
                print(f"model compressed :{compress_name}")
            else:
                if os.path.exists(model_name):
                    print(f"load:{model_name}")
                    with open(model_name, 'rb') as f:
                        esm_output = pickle.load(f)
                        del esm_output["contacts"]


                    layer = list(esm_output['representations'].keys())[0]
                    embeddings = esm_output['representations'][layer]

                    print(f"Compressing :{compress_name}")
                    with h5py.File(compress_name, "w") as f:
                        f.create_dataset("tensor", data=embeddings.numpy(), compression="gzip", compression_opts=9)

                else:
                    print(f"no model :{model_name}")


model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_8M_together_<cls>.h5
model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_8M_together_G.h5
model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_8M_together_GGGGS.h5
load:model_name
Compressing :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_8M_seperate.h5
model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_8M_seperate.h5
model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_8M_seperate.h5
model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS_35M_together_<cls>.h5
model compressed :/home/azureuser/cloudfiles/code/Users/s184243/AI4ProtEng_AIntibody/data/interim/embed_EMS

In [47]:
mypath = embed_dir
from os.path import isfile, join
from os import listdir
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

# Separate base filenames and check for missing .h5 versions
base_files = set(f for f in onlyfiles if not f.endswith(".h5"))
compressed_files = set(f for f in onlyfiles if f.endswith(".h5"))



# Find files without a compressed (.h5) version
missing_compressed = [f for f in base_files if f + ".h5" not in compressed_files]
# print(missing_compressed)
missing_compressed = [f for f in missing_compressed if "650" not in f]
missing_compressed = [f for f in missing_compressed if ".git" not in f]
missing_compressed = [f for f in missing_compressed if "cova" not in f]
missing_compressed = [f for f in missing_compressed if "3B_" not in f]
print(missing_compressed)

['embed_EMS_150_seperate', 'embed_EMS_150M_seperate', 'embed_EMS_8M_seperate', 'embed_EMS_150M_together', 'embed_EMS_8M_seperate_', 'embed_EMS_35M_seperate']
